In [1]:
#================================#
#       Lucas C. Machado         #
#       lcmachado@inf.ufrgs.br   #
#================================#

#Este trabalho foi desenvolvido para 
# a disciplina de Biologia Computacional

# O objetivo do código é comparar 
# a similaridade de dna entre
# distintos animais utilizando o algoritmo 
# de Needleman-Wunsch 

## Funções Auxiliares

In [1]:
#=======================================================
# Parâmetros que serão utilizados no cálculo das funções
#=======================================================
gap = -3
match = +2
mismatch = -2

#=======================================================
#       FUNÇÕES AUXILIARES 
#=======================================================
def matrizDeZeros(linha,coluna):
    maux = []
    for x in range(linha):
        linha = []
        for j in range(coluna):
            linha.append(0)
        maux.append(linha)
    return maux

def imprimeMatriz(mat):
    for i in range(0, len(mat)):
        for j in range(0, len(mat[i])):
            print(mat[i][j], end = "")
            if j != len(mat[i]) - 1:
                print("   ", end = "")
        print("")

def match_score(x, y):
    if x == y:
        return match
    elif x == '-' or y == '-':
        return gap
    else:
        return mismatch
    
def calcula_id(a1, a2):
    i = len(a1) -1
    id = 0
    while i >= 0:
        if(a1[i] == a2[i]):
            id +=1
        i -= 1
    return id

# le_dna_txt: Lê arquivo e devolve String contida nele
def le_dna_txt(arquivo):
    conteudo = arquivo.read()
    conteudo  = conteudo .replace('\n','')  #remover quebra de linhas
    return conteudo 

## Leitura dos Arquivos

In [3]:
# Leitura dos arquivos que contêm as sequências
arquivos = ['camundongo.txt', 'carpa_comum.txt', 'cabra.txt', 'homo_sapiens.txt']
sequencias = []

for arquivo in arquivos:
    with open(arquivo, 'r') as f:
        sequencia = le_dna_txt(f)
        sequencias.append(sequencia)

camundongo = sequencias[0]
carpa_comum = sequencias[1]
cabra = sequencias[2]
homo_sapiens = sequencias[3]


## Needleman-Wunsch

In [4]:
def needleman_wunsch_aninhamento(seq1, seq2):

    a1 = "" #string 1 
    a2 = "" #string 2 

    #tamanho das sequência
    m = len(seq1)
    n = len(seq2) 
    matriz_score = matrizDeZeros(m+1, n+1)
   
    for i in range(0, m + 1):
        matriz_score[i][0] = gap * i
    
    for j in range(0, n + 1):
        matriz_score[0][j] = gap * j
        
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            diagonal = matriz_score[i - 1][j - 1] + match_score(seq1[i-1], seq2[j-1])
            esq = matriz_score[i - 1][j] + gap
            cima = matriz_score[i][j - 1] + gap
            matriz_score[i][j] = max(cima, esq, diagonal)

    #iteradores
    i = m
    j = n
    
    while i > 0 and j > 0: 
        p = matriz_score[i][j]
        diagonal = matriz_score[i-1][j-1]
        cima = matriz_score[i][j-1]
        esq = matriz_score[i-1][j]
    
        if p == diagonal + match_score(seq1[i-1], seq2[j-1]):
            a1 += seq1[i-1]
            a2 += seq2[j-1]
            i -= 1
            j -= 1
        elif p == cima +  gap:
            a1 += '-'
            a2 += seq2[j-1]
            j -= 1
        elif p == esq +  gap:
            a1 += seq1[i-1]
            a2 += '-'
            i -= 1
    #caso as sequências não sejam do mesmo tamanho
    while i > 0:
        a1 += seq1[i-1]
        a2 += '-'
        i-=1
    while j > 0:
        a1 += '-'
        a2 += seq2[j-1]
        j-=1
    
    #Invertendo as strings que estavam invertidas (leitura começou da última posição até a posição zero)
    a1 =  a1[::-1]
    a2 =  a2[::-1]
    
    return( a1,  a2)

In [6]:
def needleman_wunsch_criaMatriz(seq1, seq2):
    
    #tamanho das sequência
    m = len(seq1)
    n = len(seq2) 
    matriz_score = matrizDeZeros(m+1, n+1)
   
    for i in range(0, m + 1):
        matriz_score[i][0] = gap * i
    
    for j in range(0, n + 1):
        matriz_score[0][j] = gap * j
        
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            diagonal = matriz_score[i - 1][j - 1] + match_score(seq1[i-1], seq2[j-1])
            esq = matriz_score[i - 1][j] + gap
            cima = matriz_score[i][j - 1] + gap
            matriz_score[i][j] = max(cima, esq, diagonal)
    return matriz_score

## Comparações de DNA 

In [7]:
#============================
# Carpa_Comum vs Homo_sapiens
#============================

seq1 = carpa_comum
seq2 = homo_sapiens

a1, a2 = needleman_wunsch_aninhamento(seq1, seq2)

score = needleman_wunsch_criaMatriz(a1,a2)[-1][-1]
print(f"Score: {score}")

identidade  = calcula_id(a1, a2) / len(a1)
print(f"Identidade: {100*round(identidade,2)}%" )

Score: 201
Identidade: 63.0%


In [8]:
#============================
# Cabra vs Homo_sapiens
#============================
seq1 = homo_sapiens
seq2 = cabra

a1, a2 = needleman_wunsch_aninhamento(seq1, seq2)
print(len(a2))
#print(a2)

score = needleman_wunsch_criaMatriz(a1,a2)[-1][-1]
print(f"Score: {score}")

identidade  = calcula_id(a1, a2) / len(a1)
print(f"Identidade: {100*round(identidade,2)}%" )

433
Score: 330
Identidade: 70.0%


In [9]:
#============================
# Carpa_comum vs Homo_sapiens
#============================
seq1 = carpa_comum
seq2 = cabra

a1, a2 = needleman_wunsch_aninhamento(seq1, seq2)

score = needleman_wunsch_criaMatriz(a1,a2)[-1][-1]
print(f"Score: {score}")

identidade  = calcula_id(a1, a2) / len(a1)
print(f"Identidade: {100*round(identidade,2)}%" )

Score: 205
Identidade: 64.0%


In [10]:
#============================
# Camundongo vs Homo_sapiens
#============================
seq1 = homo_sapiens
seq2 = camundongo

a1, a2 = needleman_wunsch_aninhamento(seq1, seq2)

score = needleman_wunsch_criaMatriz(a1,a2)[-1][-1]
print(f"Score: {score}")

identidade  = calcula_id(a1, a2) / len(a1)
print(f"Identidade: {100*round(identidade,2)}%" )

Score: 530
Identidade: 81.0%


In [11]:
#============================
# Camundongo vs Cabra
#============================
seq1 = cabra
seq2 = camundongo

a1, a2 = needleman_wunsch_aninhamento(seq1, seq2)
print(len(a2))
#print(a2)

score = needleman_wunsch_criaMatriz(a1,a2)[-1][-1]
print(f"Score: {score}")

identidade  = calcula_id(a1, a2) / len(a1)
print(f"Identidade: {100*round(identidade,2)}%" )

449
Score: 262
Identidade: 67.0%


In [12]:
#============================
# Camundongo vs Carpa-Comum
#============================
seq1 = cabra
seq2 = camundongo

a1, a2 = needleman_wunsch_aninhamento(seq1, seq2)

score = needleman_wunsch_criaMatriz(a1,a2)[-1][-1]
print(f"Score: {score}")

identidade  = calcula_id(a1, a2) / len(a1)
print(f"Identidade: {100*round(identidade,2)}%" )

Score: 262
Identidade: 67.0%
